In [ ]:
device = torch.device("mps")

In [ ]:
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

In [ ]:
# dataset, dataloader
StereoDataset = __datasets__[args.dataset]
train_dataset = StereoDataset(args.datapath, args.trainlist, True)
test_dataset = StereoDataset(args.datapath, args.testlist, False)

In [ ]:
TrainImgLoader = DataLoader(train_dataset, args.batch_size, shuffle=True, num_workers=8, drop_last=True)
TestImgLoader = DataLoader(test_dataset, args.test_batch_size, shuffle=False, num_workers=4, drop_last=False)

In [20]:
import re
def read_pfm(file_path):
    with open(file_path, 'rb') as file:
        color = None
        width = None
        height = None
        scale = None
        endian = None

        header = file.readline().rstrip()
        if header == b'PF':
            color = True
        elif header == b'Pf':
            color = False
        else:
            raise Exception('Not a PFM file.')

        dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode('latin-1'))
        if dim_match:
            width, height = map(int, dim_match.groups())
        else:
            raise Exception('Malformed PFM header.')

        scale = float(file.readline().rstrip().decode('latin-1'))
        if scale < 0:  # little-endian
            endian = '<'
            scale = -scale
        else:
            endian = '>'  # big-endian

        data = np.fromfile(file, endian + 'f')
        shape = (height, width, 3) if color else (height, width)

        data = np.reshape(data, shape)
        data = np.flipud(data)
        return data, scale

# data, scale = read_pfm(r'C:\Users\Askil\Documents\SceneFlow\sceneflow\FlyingThings\FlyingThings3D_subset\train\disparity\left\0000003.pfm')
data, scale = read_pfm(r'C:\Users\Askil\iCloudDrive\Data Science\data_science\spesiale\Sampler\FlyingThings3D\disparity\0006.pfm')
print(f"Max disparity value: {data.max()}")
print(f"Min disparity value: {data.min()}")


Max disparity value: 99.39130401611328
Min disparity value: 1.1745235919952393


In [13]:
from datasets.data_io import pfm_imread
import numpy as np

data, scale = pfm_imread(r'C:\Users\Askil\Documents\SceneFlow\sceneflow\FlyingThings\FlyingThings3D_subset\train\disparity\left\0000003.pfm')
# data = np.ascontiguousarray(data, dtype=np.float32)
print(f"Max disparity value: {data.max()}")
print(f"Min disparity value: {data.min()}")

Max disparity value: -1.1875
Min disparity value: -137.4375


In [14]:
data

array([[  -1.5625 ,   -1.5625 ,   -1.5625 , ...,   -1.375  ,   -1.375  ,
          -1.375  ],
       [  -1.5625 ,   -1.5625 ,   -1.5625 , ...,   -1.375  ,   -1.375  ,
          -1.375  ],
       [  -1.5625 ,   -1.5625 ,   -1.5625 , ...,   -1.375  ,   -1.375  ,
          -1.375  ],
       ...,
       [ -24.6875 ,  -24.6875 ,  -24.6875 , ..., -100.125  , -100.25   ,
        -100.40625],
       [ -24.6875 ,  -24.6875 ,  -24.6875 , ..., -100.09375, -100.25   ,
        -100.375  ],
       [ -24.6875 ,  -24.6875 ,  -24.6875 , ..., -100.09375, -100.21875,
        -100.34375]], dtype=float32)

In [1]:
from __future__ import print_function, division
# import argparse
import os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torchvision.utils as vutils
import torch.nn.functional as F
import numpy as np
import time
# from tensorboardX import SummaryWriter
from datasets import __datasets__
from models import __models__, model_loss_train, model_loss_test
from utils import *
from torch.utils.data import DataLoader
import gc
from dataclasses import dataclass
import matplotlib.pyplot as plt
from datasets.data_io import read_all_lines
import torchinfo
import random
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cudnn.benchmark = True
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

@dataclass
class Args:
    model: str = 'CGI_Stereo'
    maxdisp: int = 192
    dataset: str = 'sceneflow'
    # datapath: str = "/Users/askilfolgero/Documents/"
    datapath: str = r'C:\Users\Askil\Documents\SceneFlow'
    trainlist: str = Path('sceneflow_train_flying_things.txt')
    testlist: str =  Path('sceneflow_val_flying_things.txt')
    lr: float = 0.001
    batch_size: int = 1
    test_batch_size: int = 1
    epochs: int = 5
    lrepochs: str = "10,14,16,18:2"
    logdir: str = '.'
    loadckpt: str = ''
    resume: bool = False
    seed: int = 1
    summary_freq: int = 20
    save_freq: int = 1

args = Args()
torch.manual_seed(args.seed)
os.makedirs(args.logdir, exist_ok=True)

# create summary logger
# print("creating new summary file")
# logger = SummaryWriter(args.logdir)

# model, optimizer
model = __models__[args.model](args.maxdisp)
model = nn.DataParallel(model)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=(0.9, 0.999))


num_samples = 16
StereoDataset = __datasets__[args.dataset]
with open("temp_subset_filenames.txt", "w") as f:
    lines = read_all_lines(args.trainlist)
    subsample = random.choices(lines, k = num_samples)
    for sample in subsample:
        f.write(sample)
        f.write("\n")

train_dataset = StereoDataset(args.datapath, "temp_subset_filenames.txt", True)
test_dataset = StereoDataset(args.datapath, "temp_subset_filenames.txt", False)
TrainImgLoader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=8, drop_last=True)
TestImgLoader = DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False, num_workers=4, drop_last=False)


# load parameters
start_epoch = 0
if args.resume:
    # find all checkpoints file and sort according to epoch id
    all_saved_ckpts = [fn for fn in os.listdir(args.logdir) if fn.endswith(".ckpt")]
    all_saved_ckpts = sorted(all_saved_ckpts, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    # use the latest checkpoint file
    loadckpt = os.path.join(args.logdir, all_saved_ckpts[-1])
    print(f"loading the lastest model in logdir: {loadckpt}")
    state_dict = torch.load(loadckpt)
    model.load_state_dict(state_dict['model'])
    optimizer.load_state_dict(state_dict['optimizer'])
    start_epoch = state_dict['epoch'] + 1

elif args.loadckpt:
    # load the checkpoint file specified by args.loadckpt
    print("loading model {}".format(args.loadckpt))
    state_dict = torch.load(args.loadckpt)
    model_dict = model.state_dict()
    pre_dict = {k: v for k, v in state_dict['model'].items() if k in model_dict}
    model_dict.update(pre_dict) 
    # model.load_state_dict(state_dict['model'])
    model.load_state_dict(model_dict)

print(f"start at epoch {start_epoch}")


# train one sample
def train_sample(sample, compute_metrics=False):
    model.train()
    imgL        = sample['left'].to(device)
    imgR        = sample['right'].to(device)
    disp_gt     = sample['disparity'].to(device)
    disp_gt_low = sample['disparity_low'].to(device)
    print(f"Max disparity value: {disp_gt.max()}")
    print(f"Min disparity value: {disp_gt.min()}")

    optimizer.zero_grad()

    disp_ests = model(imgL, imgR)
    mask      = (disp_gt < args.maxdisp) & (disp_gt > 0)
    mask_low  = (disp_gt_low < args.maxdisp) & (disp_gt_low > 0)
    masks     = [mask, mask_low]
    disp_gts  = [disp_gt, disp_gt_low] 
    loss      = model_loss_train(disp_ests, disp_gts, masks)

    scalar_outputs = {"loss": loss}
    image_outputs = {"disp_est": disp_ests, "disp_gt": disp_gt, "imgL": imgL, "imgR": imgR}
    
    if compute_metrics:
        with torch.no_grad():
            # image_outputs["errormap"] = [disp_error_image_func()(disp_est, disp_gt) for disp_est in [disp_ests[0]]]
            scalar_outputs["D1"]      = [D1_metric(disp_est, disp_gt, mask) for disp_est in [disp_ests[0]]]
            scalar_outputs["EPE"]     = [EPE_metric(disp_est, disp_gt, mask) for disp_est in [disp_ests[0]]]
            # scalar_outputs["Thres1"]  = [Thres_metric(disp_est, disp_gt, mask, 1.0) for disp_est in [disp_ests[0]]]
            # scalar_outputs["Thres2"]  = [Thres_metric(disp_est, disp_gt, mask, 2.0) for disp_est in [disp_ests[0]]]
            # scalar_outputs["Thres3"]  = [Thres_metric(disp_est, disp_gt, mask, 3.0) for disp_est in [disp_ests[0]]]
    
    loss.backward()
    optimizer.step()

    return tensor2float(loss), tensor2float(scalar_outputs)


# test one sample
@make_nograd_func
def test_sample(sample, compute_metrics=True):
    model.eval()
    imgL    = sample['right'].to(device)
    imgR    = sample['left'].to(device)
    disp_gt = sample['disparity'].to(device)

    disp_ests = model(imgL, imgR)
    mask      = (disp_gt < args.maxdisp) & (disp_gt > 0)
    masks     = [mask]
    disp_gts  = [disp_gt]
    loss      = model_loss_test(disp_ests, disp_gts, masks)
    print(loss)

    scalar_outputs = {"loss": loss}
    # image_outputs = {"disp_est": disp_ests, "disp_gt": disp_gt, "imgL": imgL, "imgR": imgR}

    scalar_outputs["D1"]     = [   D1_metric(disp_est, disp_gt, mask) for disp_est in disp_ests]
    scalar_outputs["EPE"]    = [  EPE_metric(disp_est, disp_gt, mask) for disp_est in disp_ests]
    scalar_outputs["Thres1"] = [Thres_metric(disp_est, disp_gt, mask, 1.0) for disp_est in disp_ests]
    scalar_outputs["Thres2"] = [Thres_metric(disp_est, disp_gt, mask, 2.0) for disp_est in disp_ests]
    scalar_outputs["Thres3"] = [Thres_metric(disp_est, disp_gt, mask, 3.0) for disp_est in disp_ests]

    # if compute_metrics:
    #     image_outputs["errormap"] = [disp_error_image_func()(disp_est, disp_gt) for disp_est in disp_ests]

    return tensor2float(loss), tensor2float(scalar_outputs)


def train():
    bestepoch = 0
    error = 100
    for epoch_idx in range(start_epoch, args.epochs):
        adjust_learning_rate(optimizer, epoch_idx, args.lr, args.lrepochs)

        # training
        for batch_idx, sample in enumerate(TrainImgLoader):
            global_step = len(TrainImgLoader) * epoch_idx + batch_idx
            start_time = time.time()
            do_summary = global_step % args.summary_freq == 0
            loss, scalar_outputs = train_sample(sample, compute_metrics=do_summary)

            # if do_summary:
                # save_scalars(logger, 'train', scalar_outputs, global_step)
                # save_images(logger, 'train', image_outputs, global_step)

            del scalar_outputs
            print(f'Epoch {epoch_idx + 1}/{args.epochs}, Iter {batch_idx + 1}/{len(TrainImgLoader)}, train loss = {loss:.3f}, time = {(time.time() - start_time):.3f}')
        
        # saving checkpoints
        if (epoch_idx + 1) % args.save_freq == 0:
            checkpoint_data = {'epoch': epoch_idx, 'model': model.state_dict(), 'optimizer': optimizer.state_dict()}
            torch.save(checkpoint_data, f"{args.logdir}/checkpoint_{epoch_idx:0>6}.ckpt")
        gc.collect()

        # testing
        avg_test_scalars = AverageMeterDict()
        for batch_idx, sample in enumerate(TestImgLoader):
            global_step = len(TestImgLoader) * epoch_idx + batch_idx
            start_time = time.time()
            do_summary = global_step % args.summary_freq == 0
            loss, scalar_outputs = test_sample(sample, compute_metrics=do_summary)

            # if do_summary:
                # save_scalars(logger, 'test', scalar_outputs, global_step)
                # save_images(logger, 'test', image_outputs, global_step)
            
            avg_test_scalars.update(scalar_outputs)
            del scalar_outputs
            print(f'Epoch {epoch_idx + 1}/{args.epochs}, Iter {batch_idx + 1}/{len(TestImgLoader)}, test loss = {loss:.3f}, time = {time.time() - start_time:3f}')

        avg_test_scalars = avg_test_scalars.mean()
        nowerror = avg_test_scalars["D1"][0]

        if  nowerror < error :
            bestepoch = epoch_idx
            error = avg_test_scalars["D1"][0]

        # save_scalars(logger, 'fulltest', avg_test_scalars, len(TrainImgLoader) * (epoch_idx + 1))
        print("avg_test_scalars", avg_test_scalars)
        print('MAX epoch %d total test error = %.5f' % (bestepoch, error))
        gc.collect()

    print('MAX epoch %d total test error = %.5f' % (bestepoch, error))

c:\Users\Askil\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


start at epoch 0


In [2]:
if __name__ == '__main__':
    train()

downscale epochs: [10, 14, 16, 18], downscale rate: 2.0
setting learning rate to 0.001
Max disparity value: -1.125
Min disparity value: -91.46875


AssertionError: Empty mask, no True values

In [ ]:
torchinfo.summary(model)

In [ ]:
image_iter = iter(TestImgLoader)

In [ ]:
image = next(image_iter)

In [ ]:
output = model(image['left'][0].unsqueeze(0).cuda(), image['right'][0].unsqueeze(0).cuda())

In [ ]:
def plot_tensors(left, right, disp):
    fig, ax = plt.subplots(1, 3, figsize=(15, 15))
    ax[0].imshow(left.numpy().transpose(1, 2, 0))
    ax[0].set_title('Left Image')
    ax[1].imshow(disp.cpu().detach().squeeze(0).numpy())
    ax[1].set_title('Disparity')
    ax[2].imshow(right.numpy().transpose(1, 2, 0))
    ax[2].set_title('Right Image')
    plt.show()

# plot_tensors(image['left'][0], image['right'][0], image['disparity'][0])

In [ ]:
# Selecting the tensor and ensuring it's on CPU
tensor_gray = output[0].cpu().detach().squeeze(0).numpy()  # Removing channel dim

# Converting to numpy
image_gray = tensor_gray

plt.figure(figsize=(10, 10))
plt.imshow(image_gray, cmap='gray')
plt.axis('off')
plt.title('Grayscale Image')
plt.show()
